# Detect (with the) Model
## Step 1 - run the TFLite model locally

In this notebook, we are replicating  some of the original MobileNet TensorFlow Lite functionality.  That means taking the TensorFlow model (checkpoint) and converting it to a TensorFlow Lite model.   This is not quite what the SageMaker tutorial does.   The tutorial sticks with TensorFlow.   Confirm that the Lite model works first - because that is a known path then:
- Detect Step 2 - run endpoint as a TensorFlow model
- Detect Step 3 - run endpoint as a TensorFlow Lite Model -- the advantage of a Lite model is less latency, faster, less resources -- with slightly less accuracy.    Lite models are the path towards IoT deployments and TPU compatible models

We are testing the model before we deploy it as an endpoint.   This is optional but definitely makes sense as we learn this process

Your Python environment will require:
- tensorflow
- opencv
- if you are displaying (on a local machine - not SageMaker) - you need gtk2*x86_64

####  Installing OpenCV 4.1.2
`conda activate <your environment>`  
`pip install opencv-python`

In [1]:
import os

## Global Environment Variables

We are using objects and scripts in the project as much as possible.   They require environment variables to pass along where stuff is stored

### Note 
This notebook assumes output_tflite_graph.tflite is already on the SageMaker server (it will be if you ran the Train Step 3 notebook.)  If not, you can get it from a SageMaker Training Job /output/tflite_model on S3

### Execute the Common Globals + 1 other cell that corresponds to the model you want to use

In [2]:
# COMMON
PROJECT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(PROJECT_DIR, "data/new_jpeg_images")
ANNOTATION_DIR = os.path.join(PROJECT_DIR, "data/unverified_annotations")

# if you are running this on a remote server witout GTK, use 'None'
# if local with GTK, use 'gtk'
# DISPLAY = "None"
DISPLAY = 'gtk'

### Execute 1 of the following

In [ ]:
# MobileNet - V1 - SSD - cfa_prod (Chick-fil-A products)
MODEL_NAME = 'tf_lite'   # not using the edge_tpu 
MODEL_PATH = os.path.join(PROJECT_DIR, "tflite_model/output_tflite_graph.tflite")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")

# SOURCE images 
S3_TEST_IMAGES = "s3://cfa-eadatasciencesb-sagemaker/datasets/cfa_products/test_images/"

In [3]:
# MobileNet - V1 - SSD - coco (original)
# - this is the original model, it was downloaded to code/ckpt in TrainModel_Step1_Local.ipynb
# - convert using UnderstandingTendorRT_ConvertGraph - so now the name is the same
MODEL_NAME = 'tf_lite'   # not using the edge_tpu 
MODEL_PATH = os.path.join(PROJECT_DIR, "tflite_model/output_tflite_graph.tflite")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/ckpt/mscoco_label_map.pbtxt")

# SOURCE images 
S3_TEST_IMAGES = "s3://cfa-eadatasciencesb-sagemaker/datasets/coco/test_images/"

## Data
Copy the test images locally

### aws2
- if local, remember to update our credentials
- if using AWS CLI Version2,   
https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-linux-mac.html#cliv2-linux-mac-install  
- remember, it's:  
`aws2 s3 cp`

In [4]:
# delete the existing images
# - obviously, you can skip this cell if you don't want to delete
! rm -rf {IMAGE_DIR}

In [5]:
# version 2
! aws2 s3 cp {S3_TEST_IMAGES} {IMAGE_DIR} --recursive

download: s3://cfa-eadatasciencesb-sagemaker/datasets/coco/test_images/Sailing_02.jpg to data/new_jpeg_images/Sailing_02.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/coco/test_images/111-1152_IMG.JPG to data/new_jpeg_images/111-1152_IMG.JPG
download: s3://cfa-eadatasciencesb-sagemaker/datasets/coco/test_images/111-1122_IMG.JPG to data/new_jpeg_images/111-1122_IMG.JPG
download: s3://cfa-eadatasciencesb-sagemaker/datasets/coco/test_images/WV 200727.jpg to data/new_jpeg_images/WV 200727.jpg
download: s3://cfa-eadatasciencesb-sagemaker/datasets/coco/test_images/WV 200713.jpg to data/new_jpeg_images/WV 200713.jpg


## Detect
The detect.py script will read a directory of images, infer and generate XML (unverified) Annotations (VOC PASCAL schema).    Unverified means if you run them through a labeling program like labelImg, they do not have the verified attribute.    (Downstream, they will be ignored if you try to fold them into training data.)

In [6]:
! python code/detect.py --image_dir {IMAGE_DIR} --model_name {MODEL_NAME} --model_path {MODEL_PATH} --label_map_path {LABEL_MAP} --display {DISPLAY} --annotation_dir {ANNOTATION_DIR}

/home/jay/anaconda3/envs/tensorRT/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jay/anaconda3/envs/tensorRT/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jay/anaconda3/envs/tensorRT/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jay/anaconda3/envs/tensorRT/lib/python3.6/s

detected objects: [(1, 'person', 0.7734375, 877, 388, 1071, 886), (1, 'person', 0.71875, 1343, 686, 1468, 868)]
/home/jay/projects/ssd-dag/data/new_jpeg_images 111-1152_IMG.JPG
[(1, 'person', 0.7734375, 877, 388, 1071, 886), (1, 'person', 0.71875, 1343, 686, 1468, 868)]
/home/jay/projects/ssd-dag/data/unverified_annotations
 03:59:20  - detector - INFO - Image: /home/jay/projects/ssd-dag/data/new_jpeg_images/111-1122_IMG.JPG
 03:59:21  - detector - INFO - time spent: 0.1342
 03:59:21  - detector - INFO -       class: 4-motorcycle prob: 0.80078125  bbox: (198,548), (1376,1206)
 03:59:21  - detector - INFO -       class: 1-person prob: 0.70703125  bbox: (382,218), (981,1180)
 03:59:21  - detector - INFO -     obj detected: 2, obj ignored: 8
detected objects: [(4, 'motorcycle', 0.80078125, 198, 548, 1376, 1206), (1, 'person', 0.70703125, 382, 218, 981, 1180)]
/home/jay/projects/ssd-dag/data/new_jpeg_images 111-1122_IMG.JPG
[(4, 'motorcycle', 0.80078125, 198, 548, 1376, 1206), (1, 'person'

In [ ]:
# tarball the annotations
os.chdir("data")
! tar -czvf  unverif_annotations.tar.gz unverified_annotations

## Conclusion

If you want the annotations - you'll find the annotations tarball in data/ directory
use the Notebook browser to download it

The fastest, easiest way to review (and correct / verify) is to use labelImg program which will merge the image and annotation

The main conclusion here is our model works.